# <center> WillGraceApartment </center>

![piantina](./img/piantinaGirata.png)
![piantina](./img/Screenshot_1.png)
![piantina](./img/Screenshot_2.png)
![piantina](./img/Screenshot_3.png)

In [1]:
from pyplasm import *
import csv

Evaluating fenvs.py..
...fenvs.py imported in 0.00500736695881 seconds


In [2]:
""" intRGBColor

Web copied function to create colors from RGB
@param values: array of RGB values
@return: the color
"""
def intRGBColor(values):
        return Color4f([ values[0]/255.0,
                         values[1]/255.0,
                         values[2]/255.0,
                         1.0])
    
SILVER = intRGBColor([192,192,192])
GREY = intRGBColor([105,105,105])
GREYCYANO = intRGBColor([47,79,79])
WOOD = intRGBColor([130, 100, 1])

In [3]:
def material(ambientRGBA, diffuseRGBA, specularRGBA, emissionRGBA, shininess, pol):
    return MATERIAL([ambientRGBA[0],
              ambientRGBA[1],
              ambientRGBA[2],
              ambientRGBA[3],
              diffuseRGBA[0],
              diffuseRGBA[1],
              diffuseRGBA[2],
              diffuseRGBA[3],
              specularRGBA[0],
              specularRGBA[1],
              specularRGBA[2],
              specularRGBA[3],
              emissionRGBA[0],
              emissionRGBA[1],
              emissionRGBA[2],
              emissionRGBA[3],
              shininess
             ])(pol)




In [4]:
def readFile(file_name):
    points = []
    i = 0
    indexes = []
    with open(file_name, 'rb') as csvfile:
        buildReader = csv.reader(csvfile)
        for row in buildReader:
            points.append([float(row[0]), float(row[1])])
            points.append([float(row[2]), float(row[3])])
            i += 2
            indexes.append([i-1, i])
    return S([1])([-1])(MKPOL([points, indexes, None]))

In [5]:
"""constant var and path"""
# Path
pathExternalWalls = "./lines/external.lines"
pathInternalWalls = "./lines/internal.lines"
pathGrindWalls = "./lines/grind.lines"
pathFloor = "./lines/floor.lines"
pathHoles = "./lines/holes.lines"
pathBalconyFloor = "./lines/balconyFloor.lines"
pathDoors = "./lines/doors.lines"
pathWindows = "./lines/windows.lines"

# Constant var
externalWallsThickness = 5
internalWallsThickness = 2
grindWallsThickness = 1
grindHeight = 10
houseHeight = 40
windowsHeight = 20
windowsHeightFromFloor = 10
doorsHeight = 25

# Material Orange
ambientRGBA = [1,0,0,1]
diffuseRGBA = [0,1,0,1]
specularRGBA = [0,0,1,0]
emissionRGBA = [1,0,0,1]
shininess = 100

In [6]:
"""This function uses the constants above and creates an HPC object of an house"""

def MyAparment():
    # External Walls
    externalWallsHPC = readFile(pathExternalWalls)
    externalWallsThick = OFFSET([externalWallsThickness,externalWallsThickness])(externalWallsHPC)
    externalWallsWithOutWindows = PROD([externalWallsThick, QUOTE([houseHeight])])
    
    # Internal Walls
    internalWallsHPC = readFile(pathInternalWalls)
    internalWallsThick = OFFSET([internalWallsThickness,internalWallsThickness])(internalWallsHPC)
    internalWallDiffered = DIFF([internalWallsThick, externalWallsThick])
    internalWallsWithOutDoors = PROD([internalWallDiffered, QUOTE([houseHeight])])
    
    # Grind Walls
    grindWallsHPC = readFile(pathGrindWalls)
    grindWallsThick = OFFSET([grindWallsThickness,grindWallsThickness])(grindWallsHPC)
    grindWallDiffered = DIFF([grindWallsThick, externalWallsThick])
    grindWallsTall = PROD([grindWallDiffered, QUOTE([grindHeight])])
    
    # Windows
    windowsHPC = readFile(pathWindows)
    windowsThick = OFFSET([externalWallsThickness, externalWallsThickness])(windowsHPC)
    windowsTall = PROD([windowsThick, QUOTE([windowsHeight])])
    windowsFromFloor = STRUCT([T(3)(windowsHeightFromFloor), windowsTall])
    
    # External Wall holes
    externalWallsWithWindows = DIFF([externalWallsWithOutWindows, windowsFromFloor])
    
    #Doors
    doorsHPC = readFile(pathDoors)
    doorsExternalThick = OFFSET([externalWallsThickness+1, externalWallsThickness+1])(doorsHPC)
    doorsExternalTall = PROD([doorsExternalThick, QUOTE([doorsHeight])])
    
    # External Wall with windows and doors
    externalWallsWithWindowsAndDoors = DIFF([externalWallsWithWindows, doorsExternalTall])
    externalWallsWithWindowsAndDoorsColored = COLOR(GREY)(externalWallsWithWindowsAndDoors)
    
    grindColored = COLOR(WHITE)(grindWallsTall)
    
    # Internal Wall with windows and doors
    doorsInternalThick = OFFSET([internalWallsThickness+1, internalWallsThickness+1])(doorsHPC)
    doorsInternalTall = PROD([doorsInternalThick, QUOTE([doorsHeight])])
    internalWallsDoors = DIFF([internalWallsWithOutDoors, doorsInternalTall])
    internalWallsDoorsColored = COLOR(GREEN)(internalWallsDoors)
    
    # HoleStair
    holeHPC = readFile(pathHoles)
    holes = SOLIDIFY(holeHPC)
    
    # Floors
    floorHPC = readFile(pathFloor)
    floorHPC = SOLIDIFY(floorHPC)
    floorHPC = DIFF([floorHPC, holes])
    floorHPCColored = COLOR(WOOD)(floorHPC)
    
    # Balcony Floors
    balconyFloorHPC = readFile(pathBalconyFloor)
    balconyFloorColored = COLOR(GREEN)(SOLIDIFY(balconyFloorHPC))
    balconyMaterial = material(ambientRGBA, diffuseRGBA, specularRGBA, emissionRGBA, shininess, balconyFloorColored)
    
    # Final Struct
    struct = STRUCT([grindColored, floorHPCColored, balconyMaterial, externalWallsWithWindowsAndDoorsColored, internalWallsDoorsColored])
    
    return struct


In [7]:
VIEW(MyAparment())

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000B207D80> >